In [1]:
# Enable interactive figures in Jupyter Notebook
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import matplotlib.ticker as ticker

np.set_printoptions(suppress=True)


In [2]:
def create_rotation_matrix(yaw, pitch, roll):
    # Convert angles to radians
    yaw = np.radians(yaw)
    pitch = np.radians(pitch)
    roll = np.radians(roll)
    # Yaw matrix around Y axis
    R_yaw = np.array([
        [np.cos(yaw), 0, np.sin(yaw)],
        [0, 1, 0],
        [-np.sin(yaw), 0, np.cos(yaw)]
    ])
    # Pitch matrix around X axis
    R_pitch = np.array([
        [1, 0, 0],
        [0, np.cos(pitch), -np.sin(pitch)],
        [0, np.sin(pitch), np.cos(pitch)]
    ])
    # Roll matrix around Z axis
    R_roll = np.array([
        [np.cos(roll), -np.sin(roll), 0],
        [np.sin(roll), np.cos(roll), 0],
        [0, 0, 1]
    ])
    # Combine the matrices
    R_combined = R_roll.dot(R_pitch).dot(R_yaw)
    return R_combined

In [3]:
def createGTCameraParameters(focal_length_x, focal_length_y, principal_point_x, principal_point_y, R, camera_translation_vector_from_world_origin):
    # Construct a 4x4 extrinsic matrix
    extrinsic_matrix = np.eye(4)
    extrinsic_matrix[:3, :3] = R  # Rotation
    extrinsic_matrix[:3, 3] = -R @ camera_translation_vector_from_world_origin  # Translation
    
    # Construct the intrinsic matrix (3x4)
    K = np.array([[focal_length_x, 0, principal_point_x, 0],
                  [0, focal_length_y, principal_point_y, 0],
                  [0, 0, 1, 0]])
    
    # Camera projection matrix P = K * [R|t]
    P = np.dot(K, extrinsic_matrix)
    
    print("\nCamera Center in World Coordinates:")
    print(camera_translation_vector_from_world_origin)
    print("\nCamera Translation Vector (t):")
    print(extrinsic_matrix[:3, 3])
    print("\nCamera Rotation Matrix (R):")
    print(R)
    print("\nExtrinsic Matrix [R|t]:")
    print(extrinsic_matrix)
    print("\nIntrinsic Matrix (K):")
    print(K)
    print("\nCamera Projection Matrix (P):")
    print(P)

    return P, extrinsic_matrix, K

In [4]:
def project_world_to_camera(points_3d, P, image_width, image_height):
    # Convert 3D points to homogeneous coordinates
    points_3d_homogeneous = np.column_stack((points_3d, np.ones((points_3d.shape[0], 1))))
    # Project the points using P
    points_2d_homogeneous = np.dot(P, points_3d_homogeneous.T).T
    # Normalize homogeneous coordinates
    points_2d_normalized = points_2d_homogeneous[:, :2] / points_2d_homogeneous[:, 2:]
    # Check if points are within the image boundaries (not used for error computation)
    points_inside_frame = np.logical_and.reduce((
        points_2d_normalized[:, 0] >= 0,
        points_2d_normalized[:, 0] <= image_width,  # image width
        points_2d_normalized[:, 1] >= 0,
        points_2d_normalized[:, 1] <= image_height   # image height
    ))
    return points_2d_normalized, points_inside_frame

In [5]:
def plotWorldPoints(ax, points, rotation_matrix, camera_origin):
    num_points = len(points)
    # Pre-generate(fix) colors for consistency
    np.random.seed(0)
    colors = np.random.rand(num_points, 3)
    for i in range(num_points):
        ax.scatter(points[i, 0], points[i, 1], points[i, 2], c=[colors[i]], marker='o', s=50)
        ax.text(points[i, 0], points[i, 1], points[i, 2], f'{i+1}', color='black', fontsize=10)
    
    # Plot a red cross at the world origin
    ax.plot([0], [0], [0], marker='x', markersize=10, color='red')
    
    # Plot world coordinate axes
    world_axes_length = 1.0
    ax.plot([0, world_axes_length], [0, 0], [0, 0], color='green', label='World X')
    ax.plot([0, 0], [0, world_axes_length], [0, 0], color='orange', label='World Y')
    ax.plot([0, 0], [0, 0], [0, 3], color='purple', label='World Z')
    
    # Label the world axes
    ax.text(world_axes_length, 0, 0, 'X', color='green', fontsize=12)
    ax.text(0, world_axes_length, 0, 'Y', color='orange', fontsize=12)
    ax.text(0, 0, 3, 'Z', color='purple', fontsize=12)
    
    # Plot the camera origin and coordinate axes
    ax.plot([camera_origin[0]], [camera_origin[1]], [camera_origin[2]], marker='x', markersize=10, color='blue')
    camera_axes_length = 1.0
    camera_x_axis = rotation_matrix[:, 0] * camera_axes_length + camera_origin
    camera_y_axis = rotation_matrix[:, 1] * camera_axes_length + camera_origin
    camera_z_axis = rotation_matrix[:, 2] * 3 + camera_origin
    ax.plot([camera_origin[0], camera_x_axis[0]], [camera_origin[1], camera_x_axis[1]], [camera_origin[2], camera_x_axis[2]], color='blue', label='Camera X')
    ax.plot([camera_origin[0], camera_y_axis[0]], [camera_origin[1], camera_y_axis[1]], [camera_origin[2], camera_y_axis[2]], color='cyan', label='Camera Y')
    ax.plot([camera_origin[0], camera_z_axis[0]], [camera_origin[1], camera_z_axis[1]], [camera_origin[2], camera_z_axis[2]], color='magenta', label='Camera Z')
    ax.text(camera_x_axis[0], camera_x_axis[1], camera_x_axis[2], 'X', color='blue', fontsize=12)
    ax.text(camera_y_axis[0], camera_y_axis[1], camera_y_axis[2], 'Y', color='cyan', fontsize=12)
    ax.text(camera_z_axis[0], camera_z_axis[1], camera_z_axis[2], 'Z', color='magenta', fontsize=12)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend(loc='upper left')

    ax.grid(True)

In [6]:
def plotImagePixelPoints(ax, image_points, image_width=640, image_height=480):
    num_points = len(image_points)
    # Pre-generate(fix) colors for consistency
    np.random.seed(0)
    colors = np.random.rand(num_points, 3)
    # Invert y-coordinates for image coordinate system (origin at top-left)
    image_points[:, 1] = -image_points[:, 1]
    ax.plot([0], [0], marker='x', markersize=10, color='red')
    for i in range(num_points):
        ax.scatter(image_points[i, 0], image_points[i, 1], c=[colors[i]], marker='o', s=50)
        ax.text(image_points[i, 0], image_points[i, 1], f'{i+1}', color='black', fontsize=10)
    ax.set_xlabel('U (pixels)')
    ax.set_ylabel('V (pixels)')
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{abs(x):0.0f}'))
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: f'{abs(y):0.0f}'))
    ax.set_xlim(0, image_width)
    ax.set_ylim(-image_height, 0)
    ax.grid(True)